In [2]:
from pyspark.sql import SparkSession
import os
os.environ['JAVA_HOME'] = 'C:\\Program Files\\Java\\jdk1.8.0_202'

In [3]:
spark=SparkSession.builder.appName('ML - zero').getOrCreate()

In [4]:
df= spark.read.csv("dataset/test1.csv",header=True,inferSchema=True)

In [5]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
from pyspark.ml.feature import VectorAssembler

In [12]:
assembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent_feature")

In [14]:
assembler.transform(df).show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent_feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [15]:
transformed_df=assembler.transform(df)

In [18]:
finalized_df=transformed_df.select(["Independent_feature","Salary"])

In [19]:
finalized_df.show()

+-------------------+------+
|Independent_feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [33]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_df.randomSplit([0.7,0.3])

In [34]:
regressor=LinearRegression(featuresCol="Independent_feature",labelCol="Salary")

In [35]:
regressor=regressor.fit(train_data)

In [36]:
regressor.coefficients

DenseVector([47.619, 1285.7143])

In [37]:
regressor.intercept

13619.047619047662

In [38]:
pred_results=regressor.evaluate(test_data)

In [39]:
test_data.show()

+-------------------+------+
|Independent_feature|Salary|
+-------------------+------+
|         [23.0,2.0]| 18000|
|        [31.0,10.0]| 30000|
+-------------------+------+



In [40]:
pred_results.predictions.show()

+-------------------+------+-----------------+
|Independent_feature|Salary|       prediction|
+-------------------+------+-----------------+
|         [23.0,2.0]| 18000|17285.71428571428|
|        [31.0,10.0]| 30000|27952.38095238097|
+-------------------+------+-----------------+



In [41]:
pred_results.meanAbsoluteError

1380.9523809523762

In [42]:
pred_results.meanSquaredError

2351473.922902466